In [1]:
import torch
import os
import torch.nn as nn
from model import UNet3d

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Set the GPUs 2 and 3 to use

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())
img = torch.randn(1, 3, 52, 120, 120).to(device)
model = UNet3d(in_channels=3, n_classes=3,
                n_channels=24).to(device)
out = model(img)

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [2]:
out.shape

torch.Size([1, 3, 155, 240, 240])

In [2]:
tdata = torch.randn(1, 3, 50, 256, 256)
unet_output = unet_model(tdata)

up
torch.Size([1, 192, 50, 16, 16]) torch.Size([1, 192, 50, 32, 32])
after x1 up torch.Size([1, 192, 50, 32, 32])
after x1 pad torch.Size([1, 192, 50, 32, 32])
after concat torch.Size([1, 384, 50, 32, 32])
up
torch.Size([1, 96, 50, 32, 32]) torch.Size([1, 96, 50, 64, 64])
after x1 up torch.Size([1, 96, 50, 64, 64])
after x1 pad torch.Size([1, 96, 50, 64, 64])
after concat torch.Size([1, 192, 50, 64, 64])
up
torch.Size([1, 48, 50, 64, 64]) torch.Size([1, 48, 50, 128, 128])
after x1 up torch.Size([1, 48, 50, 128, 128])
after x1 pad torch.Size([1, 48, 50, 128, 128])
after concat torch.Size([1, 96, 50, 128, 128])
up
torch.Size([1, 24, 50, 128, 128]) torch.Size([1, 24, 50, 256, 256])
after x1 up torch.Size([1, 24, 50, 256, 256])
after x1 pad torch.Size([1, 24, 50, 256, 256])
after concat torch.Size([1, 48, 50, 256, 256])


In [5]:
from torchsummary import summary

summary(unet_model, (3,50, 256, 256))

conv: torch.Size([2, 3, 50, 256, 256]) -> torch.Size([2, 24, 50, 256, 256])
enc1: torch.Size([2, 24, 50, 256, 256]) -> torch.Size([2, 48, 50, 128, 128])
enc2: torch.Size([2, 48, 50, 128, 128]) -> torch.Size([2, 96, 50, 64, 64])
enc3: torch.Size([2, 96, 50, 64, 64]) -> torch.Size([2, 192, 50, 32, 32])
enc4: torch.Size([2, 192, 50, 32, 32]) -> torch.Size([2, 192, 50, 16, 16])
dec1: torch.Size([2, 192, 50, 16, 16]) torch.Size([2, 192, 50, 32, 32]) -> torch.Size([2, 96, 50, 32, 32])
dec2: torch.Size([2, 96, 50, 64, 64]) -> torch.Size([2, 48, 50, 64, 64])
dec3: torch.Size([2, 48, 50, 128, 128]) -> torch.Size([2, 24, 50, 128, 128])
dec4: torch.Size([2, 24, 50, 256, 256]) -> torch.Size([2, 24, 50, 256, 256])
out: torch.Size([2, 3, 50, 256, 256])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1     [-1, 24, 50, 256, 256]           1,968
         GroupNorm-2     [-1, 24, 50, 256, 256]           

RuntimeError: Numpy is not available

In [1]:
from utils import get_dataloader
from dataset import BratsDataset

test_dataloader = get_dataloader(dataset=BratsDataset, phase="test", resize_info=[0,155,3], img_width=120, data_type=['-t1c.nii.gz'], batch_size=1)

In [2]:
tb = next(iter(test_dataloader))
tb

BraTS-GLI-00421-000BraTS-GLI-00150-000BraTS-GLI-00018-000


BraTS-GLI-01484-000
BraTS-GLI-00234-000
BraTS-GLI-01197-000
BraTS-GLI-00624-000BraTS-GLI-01507-000

BraTS-GLI-01068-000


{'Id': ['BraTS-GLI-00421-000'],
 'image': tensor([[[[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]],
 
           [[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]],
 
           [[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]],
 
           ...,
 
           [[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
 